In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
from statsmodels.tsa.stattools import adfuller

# Path data yang diberikan
file_path = '/content/drive/MyDrive/data hasil Citra ke Numerik/Suhu_Permukaan_Riau.xlsx'

# Membaca data dari Excel
df = pd.read_excel(file_path)

# Membuat kolom datetime dengan cara eksplisit
df['Date'] = pd.to_datetime(df['Tahun'].astype(str) + '-' + df['Bulan'].astype(str) + '-01')

# Menetapkan kolom Date sebagai indeks
df.set_index('Date', inplace=True)

# Menyiapkan kolom 'Kota/Kabupaten' dan data suhu rata-rata
wilayah = df['Kota/Kabupaten'].unique()

# Inisialisasi list untuk menyimpan hasil differencing
results_differencing = []

# Inisialisasi DataFrame kosong untuk menyimpan data differencing
data_diff = pd.DataFrame()

# Fungsi untuk melakukan differencing dan plot sebelum dan sesudah differencing
def perform_differencing(data, kota):
    # Handle missing or infinite values
    data = data.dropna()  # Drop rows with missing values
    data = data.replace([np.inf, -np.inf], np.nan).dropna()  # Replace infinite values with NaN and drop them

    # Uji ADF sebelum differencing
    adf_statistic_before, p_value_before, _ = adfuller(data)[:3]
    stasioner_before = "Stasioner" if p_value_before < 0.05 else "Tidak Stasioner"

    # Inisialisasi jumlah differencing dan data hasil differencing
    differencing_count = 0
    differenced_data = data.copy()

    # Lakukan differencing jika data tidak stasioner
    while stasioner_before == "Tidak Stasioner" and differencing_count < 2:  # Maksimum differencing hingga 2 kali
        differencing_count += 1
        differenced_data = differenced_data.diff().dropna()  # Differencing

        # Uji ADF setelah differencing
        adf_statistic_after, p_value_after, _ = adfuller(differenced_data)[:3]
        stasioner_before = "Stasioner" if p_value_after < 0.05 else "Tidak Stasioner"

    # Simpan hasil ke dalam list
    results_differencing.append([
        kota,
        adf_statistic_before, p_value_before, "Stasioner" if p_value_before < 0.05 else "Tidak Stasioner",
        differencing_count,
        adf_statistic_after, p_value_after, "Stasioner" if p_value_after < 0.05 else "Tidak Stasioner"
    ])

    # Simpan hasil differencing ke dalam DataFrame data_diff
    if len(differenced_data) > 0:  # Hanya jika ada data yang tersisa setelah differencing
        differenced_df = pd.DataFrame({
            'Date': data.index[-len(differenced_data):],  # Pastikan panjang indeks sesuai dengan panjang data
            'Kota/Kabupaten': kota,
            'Rata-rata Suhu (°C) - Differenced': differenced_data
        }).set_index('Date')

        global data_diff
        data_diff = pd.concat([data_diff, differenced_df])

    # Plot data sebelum dan sesudah differencing
    fig, axes = plt.subplots(1, 2, figsize=(14, 6))

    # Plot sebelum differencing
    axes[0].plot(data, label=f'{kota} (Sebelum Differencing)')
    axes[0].set_title(f'Sebelum Differencing: {kota}')
    axes[0].set_xlabel('Waktu')
    axes[0].set_ylabel('Suhu (°C)')
    axes[0].xaxis.set_major_locator(mdates.YearLocator())  # Menampilkan tahun secara tahunan
    axes[0].xaxis.set_major_formatter(mdates.DateFormatter('%Y'))  # Format tahun
    axes[0].tick_params(axis='x', rotation=45)  # Memiringkan label tahun agar lebih terbaca
    axes[0].legend()
    axes[0].grid(True)

    # Plot setelah differencing
    axes[1].plot(differenced_data, label=f'{kota} (Setelah Differencing)')
    axes[1].set_title(f'Setelah Differencing: {kota}')
    axes[1].set_xlabel('Waktu')
    axes[1].set_ylabel('Suhu (°C)')
    axes[1].xaxis.set_major_locator(mdates.YearLocator())  # Menampilkan tahun secara tahunan
    axes[1].xaxis.set_major_formatter(mdates.DateFormatter('%Y'))  # Format tahun
    axes[1].tick_params(axis='x', rotation=45)  # Memiringkan label tahun agar lebih terbaca
    axes[1].legend()
    axes[1].grid(True)

    plt.tight_layout()  # Menjaga tata letak plot agar rapi
    plt.show()

# Melakukan differencing untuk setiap Kota/Kabupaten
for kota in wilayah:
    data_kota = df[df['Kota/Kabupaten'] == kota]['Rata-rata Suhu (°C)']
    perform_differencing(data_kota, kota)

# Membuat DataFrame dari hasil differencing
results_differencing_df = pd.DataFrame(results_differencing, columns=[
    'Wilayah',
    'ADF Statistic Sebelum Differencing',
    'p-value Sebelum Differencing',
    'Stasioner Sebelum Differencing',
    'Jumlah Differencing',
    'ADF Statistic Setelah Differencing',
    'p-value Setelah Differencing',
    'Stasioner Setelah Differencing'
])


Output hidden; open in https://colab.research.google.com to view.

In [ ]:
data_kota

,Rata-rata Suhu (°C)
Date,
2001-01-01,27.757637
2001-02-01,28.681278
2001-03-01,29.528048
2001-04-01,29.084145
2001-05-01,27.810105
...,...
2020-08-01,28.302988
2020-09-01,28.973096
2020-10-01,28.331944


In [ ]:
df[['Kota/Kabupaten', 'Rata-rata Suhu (°C)']]

,Kota/Kabupaten,Rata-rata Suhu (°C)
Date,,
2001-01-01,Riau,27.998061
2001-02-01,Riau,29.144357
2001-03-01,Riau,29.393144
2001-04-01,Riau,29.077206
2001-05-01,Riau,28.331078
...,...,...
2020-08-01,Siak,28.302988
2020-09-01,Siak,28.973096
2020-10-01,Siak,28.331944


In [ ]:
results_differencing_df

,Wilayah,ADF Statistic Sebelum Differencing,p-value Sebelum Differencing,Stasioner Sebelum Differencing,Jumlah Differencing,ADF Statistic Setelah Differencing,p-value Setelah Differencing,Stasioner Setelah Differencing
0,Riau,-1.605289,0.4809363442,Tidak Stasioner,1,-5.276388,0.0000061272,Stasioner
1,Bengkalis,-1.690595,0.4359645985,Tidak Stasioner,1,-9.634162,0.0000000000,Stasioner
2,Indragiri Hilir,-2.191634,0.2092957080,Tidak Stasioner,1,-5.848765,0.0000003631,Stasioner
3,Indragiri Hulu,-1.561983,0.5026574822,Tidak Stasioner,1,-10.124738,0.0000000000,Stasioner
4,Kampar,-1.237105,0.6574337794,Tidak Stasioner,1,-6.006127,0.0000001614,Stasioner
5,Kepulauan Meranti,-1.688801,0.4368917725,Tidak Stasioner,1,-3.664677,0.0046383437,Stasioner
6,Kota Dumai,-2.084027,0.2509859378,Tidak Stasioner,1,-9.003979,0.0000000000,Stasioner
7,Kota Pekanbaru,-2.633249,0.0863182356,Tidak Stasioner,1,-10.251531,0.0000000000,Stasioner
8,Kuantan Singingi,-1.684665,0.4390301279,Tidak Stasioner,1,-6.335897,0.0000000283,Stasioner
9,Pelalawan,-1.992964,0.2896666756,Tidak Stasioner,1,-9.925032,0.0000000000,Stasioner


In [ ]:
# Format p-value sebagai desimal dengan presisi 10 digit
def format_p_value(x):
    try:
        return f"{x:.10f}"
    except:
        return x  # Jika bukan angka, kembalikan nilai aslinya

# Terapkan format ke kolom p-value
results_differencing_df[['p-value Sebelum Differencing', 'p-value Setelah Differencing']] = results_differencing_df[['p-value Sebelum Differencing', 'p-value Setelah Differencing']].applymap(format_p_value)

# Menampilkan DataFrame
results_differencing_df

<ipython-input-5-56e972ee2704>:9: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  results_differencing_df[['p-value Sebelum Differencing', 'p-value Setelah Differencing']] = results_differencing_df[['p-value Sebelum Differencing', 'p-value Setelah Differencing']].applymap(format_p_value)


,Wilayah,ADF Statistic Sebelum Differencing,p-value Sebelum Differencing,Stasioner Sebelum Differencing,Jumlah Differencing,ADF Statistic Setelah Differencing,p-value Setelah Differencing,Stasioner Setelah Differencing
0,Riau,-1.605289,0.4809363442,Tidak Stasioner,1,-5.276388,0.0000061272,Stasioner
1,Bengkalis,-1.690595,0.4359645985,Tidak Stasioner,1,-9.634162,0.0000000000,Stasioner
2,Indragiri Hilir,-2.191634,0.2092957080,Tidak Stasioner,1,-5.848765,0.0000003631,Stasioner
3,Indragiri Hulu,-1.561983,0.5026574822,Tidak Stasioner,1,-10.124738,0.0000000000,Stasioner
4,Kampar,-1.237105,0.6574337794,Tidak Stasioner,1,-6.006127,0.0000001614,Stasioner
5,Kepulauan Meranti,-1.688801,0.4368917725,Tidak Stasioner,1,-3.664677,0.0046383437,Stasioner
6,Kota Dumai,-2.084027,0.2509859378,Tidak Stasioner,1,-9.003979,0.0000000000,Stasioner
7,Kota Pekanbaru,-2.633249,0.0863182356,Tidak Stasioner,1,-10.251531,0.0000000000,Stasioner
8,Kuantan Singingi,-1.684665,0.4390301279,Tidak Stasioner,1,-6.335897,0.0000000283,Stasioner
9,Pelalawan,-1.992964,0.2896666756,Tidak Stasioner,1,-9.925032,0.0000000000,Stasioner


In [ ]:
data_diff

,Kota/Kabupaten,Rata-rata Suhu (°C) - Differenced
Date,,
2001-02-01,Riau,1.146295
2001-03-01,Riau,0.248788
2001-04-01,Riau,-0.315938
2001-05-01,Riau,-0.746128
2001-06-01,Riau,0.357091
...,...,...
2020-08-01,Siak,0.543342
2020-09-01,Siak,0.670108
2020-10-01,Siak,-0.641152


In [ ]:
# Tentukan path output
output_excel_path = '/content/drive/MyDrive/data hasil Citra ke Numerik/Suhu_Permukaan_Riau_setelah_differencing.xlsx'

# Simpan DataFrame ke file Excel
data_diff.to_excel(output_excel_path, sheet_name='Differenced Data')